# Enhancing LLM Accessibility: A Deep Dive into QLoRA Through Fine-tuning Llama on a single AMD GPU

Building on the previous blog [Fine-tune Llama with LoRA blog](https://github.com/ROCm/rocm-blogs/tree/release/blogs/artificial-intelligence/llama-lora), we delve into another Parameter Efficient Fine-Tuning (PEFT) approach known as Quantized Low Rank Adaptation (QLoRA). The focus will be on leveraging QLoRA for the fine-tuning of Llama-2 using a single AMD GPU with ROCm. This task, made possible through the use of QLoRA, addresses challenges related to memory and computing limitations. The exploration aims to showcase how QLoRA can be employed to enhance accessibility to open-source large language models.

## QLoRA Fine-tuning <a class="anchor" id="Introduction"></a>
[QLoRA](https://arxiv.org/abs/2305.14314) is a fine-tuning technique that combines a high-precision computing technique with a low-precision storage method. This helps keep the model size small while making sure the model is still highly performant and accurate.

### How does QLoRA work?
In few words, QLoRA optimizes the memory usage of LLM fine-tuning without compromising performance, in contrast to standard 16-bit model fine-tuning. Specifically, QLoRA employs 4-bit quantization to compress a pretrained language model. The language model parameters are then frozen, and a modest number of trainable parameters are introduced in the form of Low-Rank Adapters. During fine-tuning, QLoRA backpropagates gradients through the frozen 4-bit quantized pretrained language model into the Low-Rank Adapters. Notably, only the LoRA layers undergo updates during training. For a more in-depth exploration of LoRA, refer to the original [LoRA](https://arxiv.org/abs/2106.09685) paper.

### QLoRA vs LoRA
QLoRA and LoRA represent two parameter-efficient fine-tuning techniques. LoRA operates as a standalone fine-tuning method, while QLoRA incorporates LoRA as an auxiliary mechanism to address errors introduced during the quantization process and to additionally minimize the resource requirements during fine-tuning.

## Step-by-step Llama 2 fine-tuning with QLoRA<a class="anchor" id="Step-By-Step-Guide"></a>
This section will guide you through the steps to fine-tune the Llama 2 model, which has 7 billion parameters, on a single AMD GPU. The key to this accomplishment lies in the crucial support of QLoRA, which plays an indispensable role in efficiently reducing memory requirements.

For that, we will use the following setup:
* Hardware & OS: See [this link](https://rocm.docs.amd.com/projects/install-on-linux/en/latest/reference/system-requirements.html) for a list of supported hardware and OS with ROCm.
* Software: 
  * [ROCm 5.7.0+](https://rocm.docs.amd.com/projects/install-on-linux/en/latest/)
  * [Pytorch for ROCm 2.0+](https://rocm.docs.amd.com/projects/install-on-linux/en/latest/how-to/3rd-party/pytorch-install.html)
* Libraries: `transformers`, `accelerate`, `peft`, `trl`, `bitsandbytes`, `scipy`

In this blog, we conducted our experiment using a single MI250GPU with the Docker image [rocm/pytorch:rocm6.1.2_ubuntu22.04_py3.10_pytorch_release-2.1.2](https://hub.docker.com/layers/rocm/pytorch/rocm6.1.2_ubuntu22.04_py3.10_pytorch_release-2.1.2/images/sha256-c8b4e8dfcc64e9bf68bf1b38a16fbc5d65b653ec600f98d3290f66e16c8b6078?context=explore).

### Step 1: Getting started

Our first step is to confirm the availability of GPU.

In [1]:
!rocm-smi --showproductname



============================ ROCm System Management Interface ============================
====================================== Product Info ======================================
GPU[0]		: Card Series: 		AMD Instinct MI300X OAM
GPU[0]		: Card Model: 		0x74a1
GPU[0]		: Card Vendor: 		Advanced Micro Devices, Inc. [AMD/ATI]
GPU[0]		: Card SKU: 		MI3SRIOV
GPU[0]		: Subsystem ID: 	0x74a1
GPU[0]		: Device Rev: 		0x00
GPU[0]		: Node ID: 		2
GPU[0]		: GUID: 		47056
GPU[0]		: GFX Version: 		gfx942
================================== End of ROCm SMI Log ===================================


We will start by installing the required libraries.

In [2]:
!pip install -q pandas peft transformers trl accelerate scipy

#### Installing bitsandbytes
ROCm needs a special version of bitsandbytes (`bitsandbytes-rocm`).
1. Install bitsandbytes using the following code.

In [3]:
%%bash
git clone --recurse https://github.com/ROCm/bitsandbytes
cd bitsandbytes
git checkout rocm_enabled
pip install -r requirements-dev.txt
cmake -DCOMPUTE_BACKEND=hip -S . #Use -DBNB_ROCM_ARCH="gfx90a;gfx942" to target specific gpu arch
make
pip install .

fatal: destination path 'bitsandbytes' already exists and is not an empty directory.
fatal: detected dubious ownership in repository at '/root/llama-Qlora/src/bitsandbytes'
To add an exception for this directory, call:

	git config --global --add safe.directory /root/llama-Qlora/src/bitsandbytes


CMake Error: The current CMakeCache.txt directory /root/llama-Qlora/src/bitsandbytes/CMakeCache.txt is different than the directory /root/m2m_nobackup/xiaosong/workspace/rocm-blogs-internal/blogs/artificial-intelligence/llama-Qlora/src/bitsandbytes where CMakeCache.txt was created. This may result in binaries being created in the wrong place. If you are not sure, reedit the CMakeCache.txt
CMake Error: The source "/root/llama-Qlora/src/bitsandbytes/CMakeLists.txt" does not match the source "/root/m2m_nobackup/xiaosong/workspace/rocm-blogs-internal/blogs/artificial-intelligence/llama-Qlora/src/bitsandbytes/CMakeLists.txt" used to generate cache.  Re-run cmake with a different source directory.
make: /opt/conda/envs/py_3.9/bin/cmake: No such file or directory
make: *** [Makefile:287: cmake_check_build_system] Error 127


Processing /root/llama-Qlora/src/bitsandbytes
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for bitsandbytes: filename=bitsandbytes-0.43.2.dev0-cp310-cp310-linux_x86_64.whl size=434347 sha256=b5b17305ebc8ec8a4ef812c99a9e98043e26458eaab345756fec8c40ece37627
  Stored in directory: /tmp/pip-ephem-wheel-cache-vnzzlspv/wheels/dc/21/2b/eed1de713fc51a96bd66a3c903986d350357c261cea7384fa9
Successfully built bitsandbytes
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.43.2.dev0
    Uninstalling bitsandbytes-0.43.2.dev0:
      Successfully uninstalled bitsandbytes-0.43.2.dev0


2. Check the bitsandbytes version.
At the time of writing this blog, the version is 0.43.0.

In [4]:
%%bash
pip list | grep bitsandbytes

bitsandbytes              0.43.2.dev0


3. Import the required packages.

In [5]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig
from trl import SFTTrainer


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/conda/envs/py_3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/envs/py_3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/envs/py_3.10/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/conda/envs/py_3.10/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  Fil

### 2. Configuring the model and data

#### Model configuration
You can access Meta's official Llama-2 model from Hugging Face after making a request, which can
take a couple of days. Instead of waiting, we'll use NousResearch’s Llama-2-7b-chat-hf as our base
model (it's the same as the original, but quicker to access).

In [6]:
# Model and tokenizer names
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
new_model_name = "Llama-2-7b-chat-hf-enhanced" #You can give your own name for fine tuned model

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  

/opt/conda/envs/py_3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


##### QLoRA 4-bit quantization configuration



As outlined in the paper, QLoRA stores weights in 4-bits, allowing computation to occur in 16 or 32-bit precision. This means whenever a QLoRA weight tensor is used, we dequantize the tensor to 16 or 32-bit precision, and then perform a matrix multiplication. Various combinations, such as float16, bfloat16, float32, etc., can be chosen. Experimentation with different 4-bit quantization variants, including normalized float 4 (NF4), or pure float4 quantization, is possible. However, guided by theoretical considerations and empirical findings from the paper, the recommendation is to opt for NF4 quantization, as it tends to deliver better performance.

In our case, we chose the following configuration:
- 4-bit quantization with NF4 type
- 16-bit (float16) for computation
- Double quantization, which uses a second quantization after the first one to save an additional 0.3 bits per parameters

Quantization parameters are controlled from the BitsandbytesConfig (see [Hugging Face documentation](https://huggingface.co/docs/transformers/main_classes/quantization#transformers.BitsAndBytesConfig)) as follows:

* Loading in 4 bits is activated through load_in_4bit
* The datatype used for quantization is specified with bnb_4bit_quant_type. Note that there are two supported quantization datatypes fp4 (four bit float) and nf4 (normal four bit float). The latter is theoretically optimal for normally distributed weights and we recommend using nf4
* The datatype used for the linear layer computations with bnb_4bit_compute_dtype
* Nested quantization is activated through bnb_4bit_use_double_quant


In [7]:
# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

In [8]:
# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map="auto"
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/envs/py_3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


#### Dataset configuration
We fine-tune our base model for a
question-and-answer task using a small data set called
[mlabonne/guanaco-llama2-1k](https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k), which
is a subset (1,000 samples) of the
[timdettmers/openassistant-guanaco](https://huggingface.co/datasets/OpenAssistant/oasst1) data set.
This data set is a human-generated, human-annotated, assistant-style conversation corpus that
contains 161,443 messages in 35 different languages, annotated with 461,292 quality ratings. This
results in over 10,000 fully annotated conversation trees.

In [9]:
# Dataset
data_name = "mlabonne/guanaco-llama2-1k"
training_data = load_dataset(data_name, split="train")
# check the data
print(training_data.shape)
# #11 is a QA sample in English
print(training_data[11])

(1000, 1)
{'text': '<s>[INST] write me a 1000 words essay about deez nuts. [/INST] The Deez Nuts meme first gained popularity in 2015 on the social media platform Vine. The video featured a young man named Rodney Bullard, who recorded himself asking people if they had heard of a particular rapper. When they responded that they had not, he would respond with the phrase "Deez Nuts" and film their reactions. The video quickly went viral, and the phrase became a popular meme. \n\nSince then, Deez Nuts has been used in a variety of contexts to interrupt conversations, derail discussions, or simply add humor to a situation. It has been used in internet memes, in popular music, and even in politics. In the 2016 US presidential election, a 15-year-old boy named Brady Olson registered as an independent candidate under the name Deez Nuts. He gained some traction in the polls and even made appearances on national news programs.\n\nThe Deez Nuts meme has had a significant impact on popular culture

### 3. Start fine-tuning

To set your training parameters, use the following code:

In [10]:
# Training Params
train_params = TrainingArguments(
    output_dir="./results_modified",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_8bit",
    save_steps=50,
    logging_steps=50,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
) 

### Training with QLoRA configuration<a class="anchor" id="Training_with_LoRA_configuration"></a>

Now you can integrate LoRA into the base model and assess its additional parameters. LoRA essentially
adds pairs of rank-decomposition weight matrices (called update matrices) to existing weights, and
only trains the newly added weights.

In [11]:
from peft import get_peft_model
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=8,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(base_model, peft_parameters)
model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


Note that there are only 0.062% parameters added by LoRA, which is a tiny portion of the original
model. This is the percentage we'll update through fine-tuning, as follows.

In [13]:
# Trainer with QLoRA configuration
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params
)

# Training
fine_tuning.train()

/opt/conda/envs/py_3.10/lib/python3.10/site-packages/peft/utils/other.py:145: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/opt/conda/envs/py_3.10/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/envs/py_3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It i

Step,Training Loss
50,1.564200
100,1.348400
150,1.277900
200,1.323500
250,1.345700


/opt/conda/envs/py_3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/envs/py_3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/envs/py_3.10/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details 

TrainOutput(global_step=250, training_loss=1.3719460601806641, metrics={'train_runtime': 312.3577, 'train_samples_per_second': 3.201, 'train_steps_per_second': 0.8, 'total_flos': 8679674339426304.0, 'train_loss': 1.3719460601806641, 'epoch': 1.0})

In [14]:
# Save Model
fine_tuning.model.save_pretrained(new_model_name)

#### Checking memory usage during training with QLoRA<a class="anchor" id="Checking_memory_usage_during_training_with_LoRA"></a>
During the training you could check the memory usage by using "rocm-smi" command in a terminal. The command will produce the following output, which tells the usage of memory and GPU.

To enhance comprehension of QLoRA's impact on training, we will conduct a quantitative analysis comparing QLoRA, LoRA, and full-parameter fine-tuning. This analysis will encompass memory usage, training speed, training loss, and other pertinent metrics, providing a comprehensive evaluation of their respective effects.

## 4. Comparison between QLoRA, LoRA, and full-parameter fine tuning <a class="anchor" id="Comparison"></a>
Building upon our earlier blog titled [Fine-tune Llama 2 with LoRA: Customizing a large language model for question-answering](https://github.com/ROCm/rocm-blogs-internal/blob/main/blogs/artificial-intelligence/llama2-lora/README.md), which demonstrated the fine-tuning of the Llama 2 model using both LoRA and full-parameter methods, we will now integrate the results obtained with QLoRA. This  aims to provide a comprehensive overview that incorporates insights from all three fine-tuning approaches.

| Metric | Full-parameter | LoRA | **QLoRA**|
|--------------------|-----------|-----------|-----------|
|Trainable parameters| 6,738,415,616 | 4,194,304 | 4,194,304 |
|Mem usage/GB |128 | 83.2 | 10.88|
|Number of GCDs |2 | 2 | 1|
|Training Speed|3 hours| 9 minutes |6 minutes|
|Training Loss|1.368|1.377|1.347|


- Memory usage: 
    - In the case of full-parameter fine-tuning, there are **6,738,415,616** trainable parameters, leading to significant memory consumption during the training back propagation stage.
    - In contrast, LoRA and QLoRA introduces only **4,194,304** trainable parameters, accounting for a mere **0.062%** of the total trainable parameters in full-parameter fine-tuning.
    - When monitoring memory usage during training, it becomes evident that fine-tuning with LoRA utilizes only 65% of the memory consumed by full-parameter fine-tuning. Impressively, QLoRA goes even further by significantly reducing memory consumption to just 8%.
    - This presents an opportunity to increase batch size, max sequence length, and train on larger datasets within the constraints of limited hardware resources.
- Training speed: 
    - The results demonstrate that full-parameter fine-tuning takes **hours** to complete, while fine-tuning with LoRA and QLoRA concludes in **minutes**.
    - Several factors contribute to this acceleration in training speed:
        - The fewer trainable parameters in LoRA translates to fewer derivative calculations and less memory needed to store and updates the weights.
        - Full-parameter fine-tuning is more prone to being memory-bound, where the data movement becomes a bottleneck for training. This is reflected in lower GPU utilization. Although adjusting training settings can alleviate this, it may require more resources (additional GPUs) and a smaller batch size.
- Accuracy: 
    - In both training sessions, a notable reduction in training loss was observed. We achieved a closely aligned training loss for three fine-tuning approaches.
    - In the original work on QLoRA, the author mentioned the performance lost due to the imprecise quantization can be fully recovered through adapter fine-tuning after quantization. In alignment with this insight, our experiments validate and resonate with this observation, emphasizing the effectiveness of adapter fine-tuning in restoring performance after the quantization process.

## 5. Test the fine-tuned model with LoRA<a class="anchor" id="Test_the_model"></a>

In [15]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
from peft import LoraConfig, PeftModel
model = PeftModel.from_pretrained(base_model, new_model_name)
model_enhanced = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
from transformers import GenerationConfig
def inference(llm_model, tokenizer, user_input):
  def formatted_prompt(question)-> str:
    return f"<|start|>user\n{question}<|end|>\n<|start|>assistant:"
  prompt = formatted_prompt(user_input)
  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6, do_sample = True,top_k=3, temperature=0.5,
                                       repetition_penalty=1.2, max_new_tokens=200, pad_token_id=tokenizer.eos_token_id
                                      )
  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
  outputs = llm_model.generate(**inputs, generation_config=generation_config)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Now we can have a test with the base model (original) and the fine-tuned model.

### Test the base model

In [17]:
inference(base_model, tokenizer, user_input='What do you think is the most important part of building an AI chatbot?')

<|start|>user
What do you think is the most important part of building an AI chatbot?<|end|>
<|start|>assistant: The most important thing when creating an AI chatbot is to make sure that it can understand and respond appropriately. This requires developing a sophisticated natural language processing (NLP) system that can accurately interpret user input, identify relevant information in its knowledge base, and generate appropriate responses based on this analysis. Additionally, ensuring that your NLP model has access to high-quality training data sets will be critical for achieving accurate results. Finally, testing with real users should always follow development so as to ensure performance under actual usage conditions before deployment!<|end|>


### Test the fine-tuned model

In [18]:
inference(model_enhanced, tokenizer, user_input='What do you think is the most important part of building an AI chatbot?')

<|start|>user
What do you think is the most important part of building an AI chatbot?<|end|>
<|start|>assistant: The most important thing to consider when creating a chatbot is its purpose. What problem does it solve, and what kind of experience are users expecting from their interactions with your bot? Once you have identified these factors, you can start thinking about how best to implement them using natural language processing (NLP) techniques such as sentiment analysis or intent detection. It’s also essential to ensure that your model works well on different devices like mobile phones or tablets so they don't break down too quickly due to limited resources available at times when people need help urgently! Lastly but not least – make sure there aren’t any security vulnerabilities left unattended after deployment by testing everything thoroughly before launching anything publically accessible online without proper authorization first because hackers might exploit those weaknesses i

You can now observe the outputs of the two models based on the given query. As anticipated, the two outputs exhibit slight differences due to the fine-tuning process altering the model weights. 

## Disclaimers

Third-party content is licensed to you directly by the third party that owns the content and is not licensed to you by AMD. ALL LINKED THIRD-PARTY CONTENT IS PROVIDED “AS IS” WITHOUT A WARRANTY OF ANY KIND. USE OF SUCH THIRD-PARTY CONTENT IS DONE AT YOUR SOLE DISCRETION AND UNDER NO CIRCUMSTANCES WILL AMD BE LIABLE TO YOU FOR ANY THIRD-PARTY CONTENT. YOU ASSUME ALL RISK AND ARE SOLELY RESPONSIBLE FOR ANY DAMAGES THAT MAY ARISE FROM YOUR USE OF THIRD-PARTY CONTENT.